In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv(r"C:\Users\Shivani\Downloads\archive (2)\movies2.csv")
df.head()

,id,original_title,overview,release_date,popularity,vote_average,vote_count,genre_names
0,278,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,1994-09-23,116.038,8.705,26221,"['Drama', 'Crime']"
1,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",1972-03-14,167.834,8.695,19877,"['Drama', 'Crime']"
2,240,The Godfather Part II,In the continuing saga of the Corleone crime f...,1974-12-20,164.908,8.576,12001,"['Drama', 'Crime']"
3,424,Schindler's List,The true story of how businessman Oskar Schind...,1993-12-15,77.039,8.567,15409,"['Drama', 'History', 'War']"
4,389,12 Angry Men,The defense and the prosecution have rested an...,1957-04-10,76.566,8.541,8276,['Drama']


In [3]:
df.shape

(9400, 8)

In [4]:
df.isnull().sum()

id                0
original_title    0
overview          1
release_date      0
popularity        0
vote_average      0
vote_count        0
genre_names       0
dtype: int64

In [9]:
df.dropna(subset=["overview"],inplace=True)
df.isnull().sum()

id                0
original_title    0
overview          0
release_date      0
popularity        0
vote_average      0
vote_count        0
genre_names       0
dtype: int64

In [11]:
df['clicked']=(df['vote_average']>=7.0).astype(int)
df['clicked'].value_counts()

clicked
0    6183
1    3216
Name: count, dtype: int64

In [13]:
#Convert genres into columns
import ast
df['genre_names'] = df['genre_names'].apply(ast.literal_eval)

# One-hot encode genres
genre_dummies = df['genre_names'].str.join('|').str.get_dummies(sep='|')

# Merge back into original DataFrame
df = pd.concat([df, genre_dummies], axis=1)
df.head()

,id,original_title,overview,release_date,popularity,vote_average,vote_count,genre_names,clicked,Action,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,278,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,1994-09-23,116.038,8.705,26221,"[Drama, Crime]",1,0,...,0,0,0,0,0,0,0,0,0,0
1,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",1972-03-14,167.834,8.695,19877,"[Drama, Crime]",1,0,...,0,0,0,0,0,0,0,0,0,0
2,240,The Godfather Part II,In the continuing saga of the Corleone crime f...,1974-12-20,164.908,8.576,12001,"[Drama, Crime]",1,0,...,0,0,0,0,0,0,0,0,0,0
3,424,Schindler's List,The true story of how businessman Oskar Schind...,1993-12-15,77.039,8.567,15409,"[Drama, History, War]",1,0,...,1,0,0,0,0,0,0,0,1,0
4,389,12 Angry Men,The defense and the prosecution have rested an...,1957-04-10,76.566,8.541,8276,[Drama],1,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df.columns

Index(['id', 'original_title', 'overview', 'release_date', 'popularity',
       'vote_average', 'vote_count', 'genre_names', 'clicked', 'Action',
       'Adventure', 'Animation', 'Comedy', 'Crime', 'Drama', 'Family',
       'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance',
       'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western'],
      dtype='object')

In [17]:
# Family-friendly: animation, children, comedy, musical
df['is_family_friendly'] = df[['Animation', 'Family', 'Comedy', 'Music']].max(axis=1)

# Romantic movies
df['is_romantic'] = df['Romance']

# Thriller-type movies
df['is_thriller'] = df[['Thriller', 'Mystery', 'Crime']].max(axis=1)

# Sci-Fi / Fantasy cluster
df['is_scifi_fantasy'] = df[['Science Fiction', 'Fantasy']].max(axis=1)

# Action-packed cluster
df['is_action_heavy'] = df[['Adventure', 'War', 'Western']].max(axis=1)

# Dark/Intense themes
df['is_dark_intense'] = df[['Horror', 'Thriller']].max(axis=1)

#Lighthearted drama
df['is_light_hearted']=df[['Drama','TV Movie']].max(axis=1)

In [19]:
cols_to_drop = [
    'Romance',  # represented by is_romantic
    'Thriller', 'Mystery', 'Horror',  # represented by is_dark_intense
    'Comedy', 'Animation',
    'TV Movie','War', 'Western','Music','Family']
df = df.drop(columns=cols_to_drop)

In [21]:
df = df.rename(columns={
    'vote_count': 'movie_rating_count',         # number of users who rated this movie
    'vote_average': 'movie_avg_rating'         # average movie rating (0–10 scale)
})
df['movie_avg_rating'] = df['movie_avg_rating'] / 2
df['popularity_norm'] = (df['popularity'] - df['popularity'].min()) / (df['popularity'].max() - df['popularity'].min())

In [23]:
df.head()

,id,original_title,overview,release_date,popularity,movie_avg_rating,movie_rating_count,genre_names,clicked,Action,...,History,Science Fiction,is_family_friendly,is_romantic,is_thriller,is_scifi_fantasy,is_action_heavy,is_dark_intense,is_light_hearted,popularity_norm
0,278,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,1994-09-23,116.038,4.3525,26221,"[Drama, Crime]",1,0,...,0,0,0,0,1,0,0,0,1,0.031966
1,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",1972-03-14,167.834,4.3475,19877,"[Drama, Crime]",1,0,...,0,0,0,0,1,0,0,0,1,0.046238
2,240,The Godfather Part II,In the continuing saga of the Corleone crime f...,1974-12-20,164.908,4.2880,12001,"[Drama, Crime]",1,0,...,0,0,0,0,1,0,0,0,1,0.045432
3,424,Schindler's List,The true story of how businessman Oskar Schind...,1993-12-15,77.039,4.2835,15409,"[Drama, History, War]",1,0,...,1,0,0,0,0,0,1,0,1,0.021221
4,389,12 Angry Men,The defense and the prosecution have rested an...,1957-04-10,76.566,4.2705,8276,[Drama],1,0,...,0,0,0,0,0,0,0,0,1,0.021091


In [25]:
df.columns

Index(['id', 'original_title', 'overview', 'release_date', 'popularity',
       'movie_avg_rating', 'movie_rating_count', 'genre_names', 'clicked',
       'Action', 'Adventure', 'Crime', 'Drama', 'Fantasy', 'History',
       'Science Fiction', 'is_family_friendly', 'is_romantic', 'is_thriller',
       'is_scifi_fantasy', 'is_action_heavy', 'is_dark_intense',
       'is_light_hearted', 'popularity_norm'],
      dtype='object')

In [27]:
df.drop(['genre_names','Science Fiction','release_date','popularity'],axis=1)

,id,original_title,overview,movie_avg_rating,movie_rating_count,clicked,Action,Adventure,Crime,Drama,Fantasy,History,is_family_friendly,is_romantic,is_thriller,is_scifi_fantasy,is_action_heavy,is_dark_intense,is_light_hearted,popularity_norm
0,278,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,4.3525,26221,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0.031966
1,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",4.3475,19877,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0.046238
2,240,The Godfather Part II,In the continuing saga of the Corleone crime f...,4.2880,12001,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0.045432
3,424,Schindler's List,The true story of how businessman Oskar Schind...,4.2835,15409,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0.021221
4,389,12 Angry Men,The defense and the prosecution have rested an...,4.2705,8276,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0.021091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9395,9544,FearDotCom,When four bodies are discovered among the indu...,1.8620,346,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0.004121
9396,485774,The Open House,A teenager and his mother find themselves besi...,1.8505,1317,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0.004030
9397,10870,Les Dalton,Joe and Averell are the eldest and youngest of...,1.8405,329,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0.005877
9398,74849,The Star Wars Holiday Special,Luke Skywalker and Han Solo battle evil Imperi...,1.6500,441,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0.003167


In [29]:
#TF-IDF
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
def clean_text(text):
    if pd.isnull(text):
        return ""
    text = text.lower()
    text = re.sub(r"\d+", "", text)                         # remove digits
    text = re.sub(r"[^\w\s]", "", text)                     # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r"\s+", " ", text).strip()                # remove extra spaces
    return text

df['overview_clean'] = df['overview'].apply(clean_text)
tfidf = TfidfVectorizer(stop_words='english', max_features=500)
overview_tfidf = tfidf.fit_transform(df['overview_clean'])
svd = TruncatedSVD(n_components=5, random_state=42)  # you can tune n_components
overview_svd = svd.fit_transform(overview_tfidf)
for i in range(overview_svd.shape[1]):
    df[f'overview_nlp_{i+1}'] = overview_svd[:, i]


In [30]:
df.drop(columns=['overview', 'overview_clean'], inplace=True)
df.columns

Index(['id', 'original_title', 'release_date', 'popularity',
       'movie_avg_rating', 'movie_rating_count', 'genre_names', 'clicked',
       'Action', 'Adventure', 'Crime', 'Drama', 'Fantasy', 'History',
       'Science Fiction', 'is_family_friendly', 'is_romantic', 'is_thriller',
       'is_scifi_fantasy', 'is_action_heavy', 'is_dark_intense',
       'is_light_hearted', 'popularity_norm', 'overview_nlp_1',
       'overview_nlp_2', 'overview_nlp_3', 'overview_nlp_4', 'overview_nlp_5'],
      dtype='object')

In [33]:
df=df.drop(columns=['Adventure','Drama'])

In [35]:
x=df.drop(['clicked','id', 'original_title','movie_avg_rating','release_date','genre_names'],axis=1)
y=df['clicked']

In [37]:
x.columns

Index(['popularity', 'movie_rating_count', 'Action', 'Crime', 'Fantasy',
       'History', 'Science Fiction', 'is_family_friendly', 'is_romantic',
       'is_thriller', 'is_scifi_fantasy', 'is_action_heavy', 'is_dark_intense',
       'is_light_hearted', 'popularity_norm', 'overview_nlp_1',
       'overview_nlp_2', 'overview_nlp_3', 'overview_nlp_4', 'overview_nlp_5'],
      dtype='object')

In [39]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.20,random_state=42)

# RandomForest

In [42]:
from sklearn.ensemble import RandomForestClassifier
rcf_model=RandomForestClassifier()
rcf_model.fit(x_train,y_train)

RandomForestClassifier()

In [43]:
y_pred_rcf=rcf_model.predict(x_test)

In [44]:
from sklearn.metrics import classification_report,accuracy_score,roc_auc_score
accuracy_rclf=accuracy_score(y_test,y_pred_rcf)
rclf=classification_report(y_test, y_pred_rcf)
rf_roc_auc=roc_auc_score(y_test,y_pred_rcf)
print("Accuracy:",accuracy_rclf)
print(rclf)
print("ROC-AUC Score:",rf_roc_auc)

Accuracy: 0.7271276595744681
              precision    recall  f1-score   support

           0       0.75      0.89      0.81      1240
           1       0.66      0.41      0.51       640

    accuracy                           0.73      1880
   macro avg       0.70      0.65      0.66      1880
weighted avg       0.72      0.73      0.71      1880

ROC-AUC Score: 0.6513860887096774


# XGBoost

In [49]:
import xgboost as xgb
from xgboost import XGBClassifier
xg_model = XGBClassifier(n_estimators=400, max_depth=8, learning_rate=0.05, eval_metric="logloss", random_state=42)
xg_model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=400,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [50]:
y_pred_xg=xg_model.predict(x_test)

In [53]:
xgb_accuracy=accuracy_score(y_test, y_pred_xg)
xgb_report=classification_report(y_test,y_pred_xg)
print("Accuracy",xgb_accuracy)
print(xgb_report)
y_proba = xg_model.predict_proba(x_test)[:, 1]
print("ROC AUC:", roc_auc_score(y_test, y_proba))

Accuracy 0.7154255319148937
              precision    recall  f1-score   support

           0       0.75      0.85      0.80      1240
           1       0.61      0.45      0.52       640

    accuracy                           0.72      1880
   macro avg       0.68      0.65      0.66      1880
weighted avg       0.70      0.72      0.70      1880

ROC AUC: 0.7339667338709677


# Save df in csv file

In [56]:
df.columns

Index(['id', 'original_title', 'release_date', 'popularity',
       'movie_avg_rating', 'movie_rating_count', 'genre_names', 'clicked',
       'Action', 'Crime', 'Fantasy', 'History', 'Science Fiction',
       'is_family_friendly', 'is_romantic', 'is_thriller', 'is_scifi_fantasy',
       'is_action_heavy', 'is_dark_intense', 'is_light_hearted',
       'popularity_norm', 'overview_nlp_1', 'overview_nlp_2', 'overview_nlp_3',
       'overview_nlp_4', 'overview_nlp_5'],
      dtype='object')

In [58]:
df=df.drop(['genre_names','Science Fiction','popularity'],axis=1)

In [60]:
df.head()

,id,original_title,release_date,movie_avg_rating,movie_rating_count,clicked,Action,Crime,Fantasy,History,...,is_scifi_fantasy,is_action_heavy,is_dark_intense,is_light_hearted,popularity_norm,overview_nlp_1,overview_nlp_2,overview_nlp_3,overview_nlp_4,overview_nlp_5
0,278,The Shawshank Redemption,1994-09-23,4.3525,26221,1,0,1,0,0,...,0,0,0,1,0.031966,0.214872,0.030931,0.139224,0.002333,-0.062085
1,238,The Godfather,1972-03-14,4.3475,19877,1,0,1,0,0,...,0,0,0,1,0.046238,0.234938,-0.063687,0.167557,0.022166,-0.082865
2,240,The Godfather Part II,1974-12-20,4.2880,12001,1,0,1,0,0,...,0,0,0,1,0.045432,0.254844,0.128699,0.072078,-0.250808,-0.207686
3,424,Schindler's List,1993-12-15,4.2835,15409,1,0,0,0,1,...,0,1,0,1,0.021221,0.202824,0.037987,-0.195274,0.519746,-0.091121
4,389,12 Angry Men,1957-04-10,4.2705,8276,1,0,0,0,0,...,0,0,0,1,0.021091,0.196590,-0.122408,-0.103768,-0.148830,-0.015802


In [62]:
# Save the DataFrame to a CSV file
df.to_csv('movies_2019_2024_ott.csv', index=False)


In [64]:
df.columns

Index(['id', 'original_title', 'release_date', 'movie_avg_rating',
       'movie_rating_count', 'clicked', 'Action', 'Crime', 'Fantasy',
       'History', 'is_family_friendly', 'is_romantic', 'is_thriller',
       'is_scifi_fantasy', 'is_action_heavy', 'is_dark_intense',
       'is_light_hearted', 'popularity_norm', 'overview_nlp_1',
       'overview_nlp_2', 'overview_nlp_3', 'overview_nlp_4', 'overview_nlp_5'],
      dtype='object')